In [1]:
import time
import stim

from surface_sim import Layout
from surface_sim.setup import CircuitNoiseSetup
from surface_sim.models import PhenomenologicalDepolNoiseModel
from surface_sim import Detectors
from surface_sim.experiments import schedule_from_circuit, experiment_from_schedule
from surface_sim.circuit_blocks.unrot_surface_code_css import gate_to_iterator
from surface_sim.layouts import unrot_surface_codes
import surface_sim.experiments.unrot_surface_code_css as exp
from qec_util.distance import get_circuit_distance

from somatching import SoMatching

# INPUTS
EXPERIMENTS = ["I", "S", "H", "CNOT-alternating", "CNOT-no-alternating"]
DISTANCES = [3, 5]
NOISE_MODEL = PhenomenologicalDepolNoiseModel
BASES = ["Z", "X"]
FRAMES = ["pre-gate", "post-gate"]

In [2]:
# GENERATE CIRCUIT
def get_num_qubits(experiment):
    return 1 if experiment in ["I", "S", "H", "SQRT_X"] else 2

def get_circuit(experiment, distance, basis):
    num_qubits = get_num_qubits(experiment)

    # reset
    circuit_str = ""
    if basis == "X":
        circuit_str = "RX " + " ".join([f"{i}" for i in range(num_qubits)]) + "\nTICK\n"
    else:
        circuit_str = "R " + " ".join([f"{i}" for i in range(num_qubits)]) + "\nTICK\n"

    # clifford gates + QEC cycles
    if num_qubits == 1:
        circuit_str += f"{experiment} 0\nTICK\n"*(distance+1)
    elif experiment == "CNOT-no-alternating":
        circuit_str += "CNOT 0 1\nTICK\n"*(distance+1)
    elif experiment == "CNOT-alternating":
        circuit_str += "CNOT 0 1\nTICK\nCNOT 1 0\nTICK\n"*((distance+1)//2)
    elif experiment == "CZ":
        circuit_str += "CZ 0 1\nTICK\n"*(distance+1)
    else:
        raise ValueError(f"{experiment} is not known.")

    # measurment
    if basis == "X":
        circuit_str += "MX " + " ".join([f"{i}" for i in range(num_qubits)])
    else:
        circuit_str += "M " + " ".join([f"{i}" for i in range(num_qubits)])

    circuit = stim.Circuit(circuit_str)

    return circuit

# SoMatching
def get_observables(experiment, basis):
    num_qubits = get_num_qubits(experiment)

    if basis == "X" and num_qubits == 1:
        return [["X0"]]
    if basis == "Z" and num_qubits == 1:
        return [["Z0"]]
    if basis == "X" and num_qubits == 2:
        return [["X0"], ["X1"]]
    if basis == "Z" and num_qubits == 2:
        return [["Z0"], ["Z1"]]
    
    raise ValueError


def get_inactive_detectors(dem_subgraph):
    inactive_dets = []
    detector_coords_def = False
    for instr in dem_subgraph.flattened():
        if instr.type == "detector":
            detector_coords_def = True
        if instr.type == "error" and detector_coords_def:
            # the inactive-detector errors appear after the detector-coords definition
            assert instr.args_copy() == [0.5]
            assert len(instr.targets_copy()) == 1
            inactive_dets.append(instr.targets_copy()[0].val)
            
    return inactive_dets

def remove_inactive_detectors_from_circuit(circuit, inactive_dets):
    new_circuit = stim.Circuit()
    det_id = 0
    for instr in circuit.flattened():
        if instr.name != "DETECTOR":
            new_circuit.append(instr)
            continue

        if det_id not in inactive_dets:
            new_circuit.append(instr)
            
        det_id += 1

    return new_circuit

def remove_inactive_logicals_from_circuit(circuit, active_logs):
    new_circuit = stim.Circuit()
    log_id = 0
    num_logs = 0
    for instr in circuit.flattened():
        if instr.name != "OBSERVABLE_INCLUDE":
            new_circuit.append(instr)
            continue

        if log_id in active_logs:
            # rewirte the logical so that the SAT solver does not complain
            new_instr = stim.CircuitInstruction(name="OBSERVABLE_INCLUDE", gate_args=[num_logs], targets=instr.targets_copy())
            new_circuit.append(new_instr)
            num_logs += 1
            
        log_id += 1

    return new_circuit

In [3]:
for experiment_name in EXPERIMENTS:
    for basis in BASES:
        for frame in FRAMES:
            for distance in DISTANCES:
                layouts = unrot_surface_codes(get_num_qubits(experiment_name), distance=distance)
                qubit_inds = {}
                anc_coords = {}
                anc_qubits = []
                stab_coords = {}
                for l, layout in enumerate(layouts):
                    qubit_inds.update(layout.qubit_inds())
                    anc_qubits += layout.get_qubits(role="anc")
                    coords = layout.anc_coords()
                    anc_coords.update(coords)
                    stab_coords[f"Z{l}"] = [v for k,v in coords.items() if k[0] == "Z"]
                    stab_coords[f"X{l}"] = [v for k,v in coords.items() if k[0] == "X"]

                circuit = get_circuit(experiment_name, distance, basis)

                setup = CircuitNoiseSetup()
                setup.set_var_param("prob", 1e-3)
                model = NOISE_MODEL(setup=setup, qubit_inds=qubit_inds)
                detectors = Detectors(anc_qubits, frame=frame, anc_coords=anc_coords)

                schedule = schedule_from_circuit(circuit, layouts, gate_to_iterator)
                experiment = experiment_from_schedule(
                    schedule, model, detectors, anc_reset=True, anc_detectors=None
                )
                
                dem = experiment.detector_error_model(allow_gauge_detectors=True)
                decoder = SoMatching(dem, circuit, get_observables(experiment_name, basis), stab_coords, frame)

                print("start=", time.strftime("%D %H:%M:%S"))
                d_circ_list = []
                for i in range(get_num_qubits(experiment_name)):
                    inactive_dets = get_inactive_detectors(decoder.decoding_subgraphs[i])
                    experiment_subgraph = remove_inactive_detectors_from_circuit(experiment, inactive_dets=inactive_dets)
                    experiment_subgraph = remove_inactive_logicals_from_circuit(experiment_subgraph, active_logs=[i])
                    d_circ = get_circuit_distance(experiment_subgraph)
                    d_circ_list.append(d_circ)
                print("finish=", time.strftime("%D %H:%M:%S"))
                    
                print(f"{experiment_name} b={basis} f={frame} d={distance} d_circ_subgraph={d_circ_list}")

start= 02/13/25 16:21:54
finish= 02/13/25 16:21:55
I b=Z f=pre-gate d=3 d_circ_subgraph=[3]
start= 02/13/25 16:21:56
finish= 02/13/25 16:21:56
I b=Z f=pre-gate d=5 d_circ_subgraph=[5]
start= 02/13/25 16:21:56
finish= 02/13/25 16:21:56
I b=Z f=post-gate d=3 d_circ_subgraph=[3]
start= 02/13/25 16:21:58
finish= 02/13/25 16:21:58
I b=Z f=post-gate d=5 d_circ_subgraph=[5]
start= 02/13/25 16:21:58
finish= 02/13/25 16:21:58
I b=X f=pre-gate d=3 d_circ_subgraph=[3]
start= 02/13/25 16:21:59
finish= 02/13/25 16:22:00
I b=X f=pre-gate d=5 d_circ_subgraph=[5]
start= 02/13/25 16:22:00
finish= 02/13/25 16:22:00
I b=X f=post-gate d=3 d_circ_subgraph=[3]
start= 02/13/25 16:22:01
finish= 02/13/25 16:22:01
I b=X f=post-gate d=5 d_circ_subgraph=[5]
start= 02/13/25 16:22:02
finish= 02/13/25 16:22:02
S b=Z f=pre-gate d=3 d_circ_subgraph=[3]
start= 02/13/25 16:22:03
finish= 02/13/25 16:22:03
S b=Z f=pre-gate d=5 d_circ_subgraph=[5]
start= 02/13/25 16:22:04
finish= 02/13/25 16:22:04
S b=Z f=post-gate d=3 d_c

In [4]:
EXPERIMENTS = ["I", "S", "H", "CNOT-alternating", "CNOT-no-alternating"]
DISTANCES = [7]
NOISE_MODEL = PhenomenologicalDepolNoiseModel
BASES = ["Z", "X"]
FRAMES = ["pre-gate", "post-gate"]

for experiment_name in EXPERIMENTS:
    for basis in BASES:
        for frame in FRAMES:
            for distance in DISTANCES:
                layouts = unrot_surface_codes(get_num_qubits(experiment_name), distance=distance)
                qubit_inds = {}
                anc_coords = {}
                anc_qubits = []
                stab_coords = {}
                for l, layout in enumerate(layouts):
                    qubit_inds.update(layout.qubit_inds())
                    anc_qubits += layout.get_qubits(role="anc")
                    coords = layout.anc_coords()
                    anc_coords.update(coords)
                    stab_coords[f"Z{l}"] = [v for k,v in coords.items() if k[0] == "Z"]
                    stab_coords[f"X{l}"] = [v for k,v in coords.items() if k[0] == "X"]

                circuit = get_circuit(experiment_name, distance, basis)

                setup = CircuitNoiseSetup()
                setup.set_var_param("prob", 1e-3)
                model = NOISE_MODEL(setup=setup, qubit_inds=qubit_inds)
                detectors = Detectors(anc_qubits, frame=frame, anc_coords=anc_coords)

                schedule = schedule_from_circuit(circuit, layouts, gate_to_iterator)
                experiment = experiment_from_schedule(
                    schedule, model, detectors, anc_reset=True, anc_detectors=None
                )
                
                dem = experiment.detector_error_model(allow_gauge_detectors=True)
                decoder = SoMatching(dem, circuit, get_observables(experiment_name, basis), stab_coords, frame)

                print("start=", time.strftime("%D %H:%M:%S"))
                d_circ_list = []
                for i in range(get_num_qubits(experiment_name)):
                    inactive_dets = get_inactive_detectors(decoder.decoding_subgraphs[i])
                    experiment_subgraph = remove_inactive_detectors_from_circuit(experiment, inactive_dets=inactive_dets)
                    experiment_subgraph = remove_inactive_logicals_from_circuit(experiment_subgraph, active_logs=[i])
                    d_circ = get_circuit_distance(experiment_subgraph)
                    d_circ_list.append(d_circ)
                print("finish=", time.strftime("%D %H:%M:%S"))
                    
                print(f"{experiment_name} b={basis} f={frame} d={distance} d_circ_subgraph={d_circ_list}")

start= 02/13/25 16:23:15
finish= 02/13/25 16:23:23
I b=Z f=pre-gate d=7 d_circ_subgraph=[7]
start= 02/13/25 16:23:26
finish= 02/13/25 16:23:34
I b=Z f=post-gate d=7 d_circ_subgraph=[7]
start= 02/13/25 16:23:38
finish= 02/13/25 16:23:52
I b=X f=pre-gate d=7 d_circ_subgraph=[7]
start= 02/13/25 16:23:56
finish= 02/13/25 16:24:10
I b=X f=post-gate d=7 d_circ_subgraph=[7]
start= 02/13/25 16:24:14
finish= 02/13/25 16:24:22
S b=Z f=pre-gate d=7 d_circ_subgraph=[7]
start= 02/13/25 16:24:26
finish= 02/13/25 16:24:33
S b=Z f=post-gate d=7 d_circ_subgraph=[7]
start= 02/13/25 16:24:37
finish= 02/13/25 17:18:51
S b=X f=pre-gate d=7 d_circ_subgraph=[7]
start= 02/13/25 17:18:55
finish= 02/13/25 19:50:17
S b=X f=post-gate d=7 d_circ_subgraph=[7]
start= 02/13/25 19:50:20
finish= 02/13/25 19:50:31
H b=Z f=pre-gate d=7 d_circ_subgraph=[7]
start= 02/13/25 19:50:35
finish= 02/13/25 19:50:41
H b=Z f=post-gate d=7 d_circ_subgraph=[7]
start= 02/13/25 19:50:45
finish= 02/13/25 19:51:02
H b=X f=pre-gate d=7 d_c